In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB

/home/awanish/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
#train.A.unique()

In [3]:
train.head()
#train.drop(['A','I','J','L'],axis=1).head()

,id,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P
0,1,b,18.42,10.415,y,p,aa,v,0.125,t,f,0,f,g,120.0,375,1
1,2,a,21.75,11.750,u,g,c,v,0.250,f,f,0,t,g,180.0,0,1
2,3,b,30.17,1.085,y,p,c,v,0.040,f,f,0,f,g,170.0,179,1
3,4,b,22.67,2.540,y,p,c,h,2.585,t,f,0,f,g,0.0,0,0
4,5,a,36.00,1.000,u,g,c,v,2.000,t,t,11,f,g,0.0,456,0


In [4]:
test.head()
#test.drop(['A','I','J','L'],axis=1).head()

,id,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
0,553,b,29.17,3.500,u,g,w,v,3.500,t,t,3,t,g,329.0,0
1,554,b,28.33,5.000,u,g,w,v,11.000,t,f,0,t,g,70.0,0
2,555,b,29.25,14.790,u,g,aa,v,5.040,t,t,5,t,g,168.0,0
3,556,a,44.17,6.665,u,g,q,v,7.375,t,t,3,t,g,0.0,0
4,557,b,18.08,5.500,u,g,k,v,0.500,t,f,0,f,g,80.0,0


In [5]:
columns_list=['A','D','E','F','G','I','J','L','M']
def encode_df(dataframe):
    le = LabelEncoder()
    for column in columns_list:
        dataframe[column] = le.fit_transform(dataframe[column].astype(str))
    return dataframe

#encode the train dataframe
df_train=encode_df(train)
df_train=df_train[df_train.isnull().any(axis=1)==False]#filter all null values
#encode the test dataframe
df_test=encode_df(test)
#df_test=df_test[df_test.isnull().any(axis=1)==False]
print(df_test.shape)
s1=df_test['id']
df_train.set_index('id',inplace=True)#setting index
df_test.set_index('id',inplace=True)#setting index
df_test.head()

(138, 16)


,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
id,,,,,,,,,,,,,,,
553,1,29.17,3.500,1,0,13,7,3.500,1,1,3,1,0,329.0,0
554,1,28.33,5.000,1,0,13,7,11.000,1,0,0,1,0,70.0,0
555,1,29.25,14.790,1,0,0,7,5.040,1,1,5,1,0,168.0,0
556,0,44.17,6.665,1,0,11,7,7.375,1,1,3,1,0,0.0,0
557,1,18.08,5.500,1,0,8,7,0.500,1,0,0,0,0,80.0,0


In [6]:
# #Nomalisation
# nomalise_column=['B','C','G','H','N','O']
# norm_test_column=['B','C','F','H','K','N']
# scaler = preprocessing.StandardScaler()
# #scaler1 = preprocessing.StandardScaler()
# df_train[nomalise_column] = pd.DataFrame(scaler.fit_transform(df_train[nomalise_column]))
# #df_test[norm_test_column] = pd.DataFrame(scaler1.fit_transform(df_test[norm_test_column]))
# df_train.head()

#features and labels
y=df_train['P']
X_train=df_train.iloc[:,0:15]
df_test.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
id,,,,,,,,,,,,,,,
553,1,29.17,3.500,1,0,13,7,3.500,1,1,3,1,0,329.0,0
554,1,28.33,5.000,1,0,13,7,11.000,1,0,0,1,0,70.0,0
555,1,29.25,14.790,1,0,0,7,5.040,1,1,5,1,0,168.0,0
556,0,44.17,6.665,1,0,11,7,7.375,1,1,3,1,0,0.0,0
557,1,18.08,5.500,1,0,8,7,0.500,1,0,0,0,0,80.0,0


In [7]:
X_train.fillna(X_train.mean(), inplace=True)
y.fillna(y.mean(), inplace=True)
df_test.fillna(df_test.mean(), inplace=True)
print(X_train.shape,y.shape,df_test.shape)

(533, 15) (533,) (138, 15)


/home/awanish/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [8]:
df_test.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O
id,,,,,,,,,,,,,,,
553,1,29.17,3.500,1,0,13,7,3.500,1,1,3,1,0,329.0,0
554,1,28.33,5.000,1,0,13,7,11.000,1,0,0,1,0,70.0,0
555,1,29.25,14.790,1,0,0,7,5.040,1,1,5,1,0,168.0,0
556,0,44.17,6.665,1,0,11,7,7.375,1,1,3,1,0,0.0,0
557,1,18.08,5.500,1,0,8,7,0.500,1,0,0,0,0,80.0,0


In [23]:
pl1 = Pipeline([
         ('scale', StandardScaler()),
         ('dim_red', PCA(n_components=X_train.shape[1])), #TruncatedSVD(n_components=train_in.shape[1])),
         ('normalizer', Normalizer(norm='l2')),
         ('best_feat', SelectKBest()),
         ('clf', RandomForestClassifier(n_estimators=20))
        ])

In [24]:
scores = cross_val_score(pl1, X_train, y, cv=5)
print("RF Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

RF Accuracy: 0.85 (+/- 0.07)


In [27]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier()
clf3 = GaussianNB()
clf4 = SVC()

eclf = VotingClassifier(estimators=[('lr', clf1),
                                    ('rf', clf2),
                                    ('gnb', clf3),
                                    ('svm', clf4)])

for ki in range(6,12):
    pl = Pipeline([
             ('scale', StandardScaler()),
             #~ ('normalizer', Normalizer(norm='l2')),
             ('dim_red', PCA(n_components=X_train.shape[1])),
             #~ ('dim_red', TruncatedSVD(n_components=X.shape[1]-1)),
             ('best_feat', SelectKBest(k=ki)),
             ('clf', eclf)
            ])
    
    scores = cross_val_score(pl, X_train, y, cv=5)
    print("Ensemble | k %d | Accuracy: %0.2f (+/- %0.2f)" % (ki, scores.mean(), scores.std() * 2))

Ensemble | k 6 | Accuracy: 0.86 (+/- 0.06)
Ensemble | k 7 | Accuracy: 0.87 (+/- 0.04)
Ensemble | k 8 | Accuracy: 0.86 (+/- 0.06)
Ensemble | k 9 | Accuracy: 0.86 (+/- 0.05)
Ensemble | k 10 | Accuracy: 0.86 (+/- 0.05)
Ensemble | k 11 | Accuracy: 0.87 (+/- 0.04)


In [12]:
pl.fit(X_train,y)

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('dim_red', PCA(copy=True, iterated_power='auto', n_components=15, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('best_feat', SelectKBest(k=11, score_func=<function f_classif at 0x7fba900de7b8>)), ('clf',...ne, shrinking=True,
  tol=0.001, verbose=False))],
         n_jobs=1, voting='hard', weights=None))])

In [13]:
y_pred=pl.predict(df_test)
predictions = [value for value in y_pred]
np.unique(predictions)

array([0, 1])

In [14]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0])

In [15]:
se=pd.Series(predictions)
se.unique()

array([0, 1])

In [16]:
print(s1.shape,se.shape)

(138,) (138,)


In [17]:
final = s1.to_frame().join(se.to_frame(name='P'))

In [18]:
final

,id,P
0,553,0
1,554,0
2,555,0
3,556,0
4,557,0
5,558,0
6,559,0
7,560,0
8,561,0
9,562,1


In [19]:
np.unique(final)

array([  0,   1, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563,
       564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576,
       577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589,
       590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602,
       603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615,
       616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628,
       629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641,
       642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654,
       655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667,
       668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680,
       681, 682, 683, 684, 685, 686, 687, 688, 689, 690])

In [20]:
np.unique(y_pred)

array([0, 1])

In [21]:
#final.to_csv('result_v03',encoding='utf-8', index=False)

In [22]:
final.columns

Index(['id', 'P'], dtype='object')